# Data Import & Preprocessing

In [ ]:
!pip install feature_engine
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from feature_engine.creation import CyclicalFeatures
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import warnings
import plotly.graph_objects as go
from scipy.spatial.distance import pdist, cdist, squareform
from scipy.linalg import cholesky, cho_solve
import math
warnings.filterwarnings(action='ignore')
# np.random.seed(41)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
fileName = "/content/sample_data/Dataset.xlsx"
df_Labels = pd.read_excel(fileName, sheet_name="Plant_1_Data")
df_Labels = df_Labels.drop(['PLANT_ID'], axis=1)
df_Labels["DATE_TIME"] = pd.to_datetime(df_Labels.DATE_TIME)
df_Features = pd.read_excel(fileName, sheet_name="Plant_1_Sensor_Data")
df_Features = df_Features.drop(['PLANT_ID','SOURCE_KEY'], axis=1)
df_Features['DATE_TIME'] = pd.to_datetime(df_Features.DATE_TIME)
print(df_Features.head())

            DATE_TIME  AMBIENT_TEMPERATURE  MODULE_TEMPERATURE  IRRADIATION
0 2020-05-15 00:00:00            25.184316           22.857507          0.0
1 2020-05-15 00:15:00            25.084589           22.761668          0.0
2 2020-05-15 00:30:00            24.935753           22.592306          0.0
3 2020-05-15 00:45:00            24.846130           22.360852          0.0
4 2020-05-15 01:00:00            24.621525           22.165423          0.0


In [ ]:
df_Learn = pd.merge(df_Features, df_Labels, how='inner', left_on = ['DATE_TIME'], right_on = ['DATE_TIME'])

In [ ]:
X = df_Learn.drop(['DC_POWER', 'AC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD'], axis = 1)
X['hour'] = X['DATE_TIME'].dt.hour
X['min'] = X['DATE_TIME'].dt.minute
X = X.drop(['DATE_TIME'], axis = 1)
y = np.log(df_Learn['DC_POWER'] + 1)
display(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=44)
cols_to_scale = ['AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION']
scaler = StandardScaler()
scaler.fit(X_train[cols_to_scale])
X_train[cols_to_scale] = scaler.transform(X_train[cols_to_scale])
X_test[cols_to_scale] = scaler.transform(X_test[cols_to_scale])
display(X_train)

,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,SOURCE_KEY,hour,min
0,25.184316,22.857507,0.0,1BY6WEcLGh8j5v7,0,0
1,25.184316,22.857507,0.0,1IF53ai7Xc0U56Y,0,0
2,25.184316,22.857507,0.0,3PZuoBAID5Wc2HD,0,0
3,25.184316,22.857507,0.0,7JYdWkrLSPkdwr4,0,0
4,25.184316,22.857507,0.0,McdE0feGgRqW7Ca,0,0
...,...,...,...,...,...,...
45675,21.909288,20.427972,0.0,uHbuxQJl8lW7ozc,23,45
45676,21.909288,20.427972,0.0,wCURE6d3bPkepu2,23,45
45677,21.909288,20.427972,0.0,z9Y9gH1T5YWrNuG,23,45
45678,21.909288,20.427972,0.0,zBIq5rxdHJRwDNY,23,45


,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,SOURCE_KEY,hour,min
19883,-0.414164,-0.148314,0.144993,ih0vzX44oOqAx2f,7,45
3891,-0.491461,-0.810794,-0.773558,7JYdWkrLSPkdwr4,0,0
17244,-0.479905,-0.800226,-0.773558,WRmjgnKYAwPKWDb,1,15
18433,2.103800,1.627939,1.441359,YxYtjZvoooNbGkE,14,45
26162,0.002767,0.573704,0.687787,iCRJl6heRkivqQ3,8,15
...,...,...,...,...,...,...
19183,-0.489952,-0.708096,-0.773558,ZoEaEvLYb1n2sOq,23,15
4180,-0.759881,-0.824044,-0.773558,WRmjgnKYAwPKWDb,3,15
25773,-0.903638,-0.845808,-0.773558,z9Y9gH1T5YWrNuG,3,45
3491,0.571729,-0.528682,-0.773558,zVJPv84UY57bAof,19,15


# Feature Manipulation

In [ ]:
cyclical = CyclicalFeatures(variables=['hour', 'min'], drop_original=True)
display(X_train)
cyclical.fit(X_train)
X_train = cyclical.transform(X_train)
X_test = cyclical.transform(X_test)

display(X_train, y_train)

,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,SOURCE_KEY,hour,min
19883,-0.414164,-0.148314,0.144993,ih0vzX44oOqAx2f,7,45
3891,-0.491461,-0.810794,-0.773558,7JYdWkrLSPkdwr4,0,0
17244,-0.479905,-0.800226,-0.773558,WRmjgnKYAwPKWDb,1,15
18433,2.103800,1.627939,1.441359,YxYtjZvoooNbGkE,14,45
26162,0.002767,0.573704,0.687787,iCRJl6heRkivqQ3,8,15
...,...,...,...,...,...,...
19183,-0.489952,-0.708096,-0.773558,ZoEaEvLYb1n2sOq,23,15
4180,-0.759881,-0.824044,-0.773558,WRmjgnKYAwPKWDb,3,15
25773,-0.903638,-0.845808,-0.773558,z9Y9gH1T5YWrNuG,3,45
3491,0.571729,-0.528682,-0.773558,zVJPv84UY57bAof,19,15


,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,SOURCE_KEY,hour_sin,hour_cos,min_sin,min_cos
19883,-0.414164,-0.148314,0.144993,ih0vzX44oOqAx2f,9.422609e-01,-0.334880,-2.449294e-16,1.0
3891,-0.491461,-0.810794,-0.773558,7JYdWkrLSPkdwr4,0.000000e+00,1.000000,0.000000e+00,1.0
17244,-0.479905,-0.800226,-0.773558,WRmjgnKYAwPKWDb,2.697968e-01,0.962917,8.660254e-01,-0.5
18433,2.103800,1.627939,1.441359,YxYtjZvoooNbGkE,-6.310879e-01,-0.775711,-2.449294e-16,1.0
26162,0.002767,0.573704,0.687787,iCRJl6heRkivqQ3,8.169699e-01,-0.576680,8.660254e-01,-0.5
...,...,...,...,...,...,...,...,...
19183,-0.489952,-0.708096,-0.773558,ZoEaEvLYb1n2sOq,-2.449294e-16,1.000000,8.660254e-01,-0.5
4180,-0.759881,-0.824044,-0.773558,WRmjgnKYAwPKWDb,7.308360e-01,0.682553,8.660254e-01,-0.5
25773,-0.903638,-0.845808,-0.773558,z9Y9gH1T5YWrNuG,7.308360e-01,0.682553,-2.449294e-16,1.0
3491,0.571729,-0.528682,-0.773558,zVJPv84UY57bAof,-8.878852e-01,0.460065,8.660254e-01,-0.5


19883    7.990026
3891     0.000000
17244    0.000000
18433    9.105063
26162    8.666770
           ...   
19183    0.000000
4180     0.000000
25773    0.000000
3491     0.000000
14100    9.110921
Name: DC_POWER, Length: 41112, dtype: float64

# Model Training

In [ ]:
X_train = X_train.drop(['SOURCE_KEY'], axis=1)
X_test = X_test.drop(['SOURCE_KEY'], axis=1)
kern = ConstantKernel(constant_value=1.0, constant_value_bounds=(0.0, 10.0)) * RBF(length_scale=0.5, length_scale_bounds=(0.0, 10.0)) + RBF(length_scale=2.0, length_scale_bounds=(0.0, 10.0))
kern = RBF(length_scale=0.5, length_scale_bounds=(0.0, 10.0)) + RBF(length_scale=2.0, length_scale_bounds=(0.0, 10.0))
gaussian_process_regr_model = GaussianProcessRegressor(alpha=0.001, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=0, normalize_y=False, copy_X_train=False)
default_model = GaussianProcessRegressor(kernel=None, alpha=1e-10, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=0, normalize_y=False, copy_X_train=True)
gaussian_process_regr_model.fit(X_train[:5000], y_train[:5000])
default_model.fit(X_train[:5000], y_train[:5000])
y_pred = gaussian_process_regr_model.predict(X_test)
y_pred_def = default_model.predict(X_test)
print("Score for default method: " + str(default_model.score(X_test, y_test)))
print("Score for proposed method: " + str(gaussian_process_regr_model.score(X_test, y_test)))

Score for default method: -11.836539520986305
Score for proposed method: 0.9934667779448321


In [ ]:
vals = pd.DataFrame([[pd.to_datetime('2020-05-15 12:30:00'), '32.14768473',	'52.35325513',	'0.6492476293', 'HmiyD2TTLFNqkNe']], columns=('DATE_TIME', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION', 'SOURCE_KEY'))
vals['hour'] = vals['DATE_TIME'].dt.hour
vals['min'] = vals['DATE_TIME'].dt.minute
vals = vals.drop(['DATE_TIME'], axis = 1)
display(vals)
vals[cols_to_scale] = scaler.transform(vals[cols_to_scale])
#display(vals)
vals = cyclical.transform(vals)
predicted_value = np.exp(gaussian_process_regr_model.predict(vals.drop(['SOURCE_KEY'], axis = 1))) - 1
if predicted_value < 0:
  predicted_value = 0.0
display("DC_POWER Prediction for sample: " + str(predicted_value))

,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,SOURCE_KEY,hour,min
0,32.14768473,52.35325513,0.6492476293,HmiyD2TTLFNqkNe,12,30


'DC_POWER Prediction for sample: [9059.39900722]'

In [ ]:
pred = default_model.predict(X_test)
pred[pred < 0] = 0.0
mae = mean_absolute_error(y_test, pred)
print("Mean Absolute Error for default method: "+ str(mae))

pred = gaussian_process_regr_model.predict(X_test)
pred[pred < 0] = 0.0
mae = mean_absolute_error(y_test, pred)
print("Mean Absolute Error for proposed method: "+ str(mae))

Mean Absolute Error for default method: 0.4440581388508883
Mean Absolute Error for proposed method: 0.12933252139433513


# Significance Tests
Two tailed Student-T Test for y_test and pred with the number of samples in each being 4568 at a 95% significance level. We choose the T Test since population variance is unknown.

In [ ]:
display("Size of samples: " + str(len(y_test)))
#Variance check for two tailed test
display("Variance of Predicted Samples: " + str(np.var(np.exp(pred) - 1)), "Variance of Ground Truth: " + str(np.var(np.exp(y_test)) - 1))

#Ratio
display("Ratio to check for nearly equal Variance: " + str(np.var(pred)/np.var(y_test)))

'Size of samples: 4568'

'Variance of Predicted Samples: 16342198.026563346'

'Variance of Ground Truth: 16867028.96487056'

'Ratio to check for nearly equal Variance: 0.9804183305003554'

In [ ]:
#T-Test
pred1 = default_model.predict(X_test)
tstat, t_pval = stats.ttest_ind(a=pred1, b=y_test, equal_var=True)
display("T-Statistic for default method: " + str(tstat.round(3)), "PValue: " + str(t_pval.round(3)))

pred2 = gaussian_process_regr_model.predict(X_test)
tstat, t_pval = stats.ttest_ind(a=pred2, b=y_test, equal_var=True)
display("T-Statistic for proposed method: " + str(tstat.round(3)), "PValue: " + str(t_pval.round(3)))

'T-Statistic for default method: -2.045'

'PValue: 0.041'

'T-Statistic for proposed method: -0.12'

'PValue: 0.905'

With such a high P-value >> α=0.025, we cannot reject the null hypothesis, the means of the two samples are equal.

In [ ]:
class GPR:
  def __init__(self, length_scale=1):
    self.length_scale = length_scale
    self.kernel = rbf_kernel

  def fit(self, X, y):
    self.kern = self.kernel(i=X, length_scale=self.length_scale)
    lower = True
    L = cholesky(self.kern, lower=True)
    self.alpha = cho_solve((L, lower), y)
    self.X_train = X
    self.L = L

  def predict(self, X):
    K_star = self.kernel(X, self.X_train, length_scale=self.length_scale)
    y_mean = K_star.dot(self.alpha)
    lower = True
    v = cho_solve((self.L, lower), K_star.T)
    y_cov = self.kernel(X, length_scale=self.length_scale) - K_star.dot(v)
    return y_mean

def rbf_kernel(i, j=None, length_scale=1):
  if j is None:
    dist = pdist(i/length_scale, metric="sqeuclidean")
    K = np.exp(-0.5 * dist)
    K = squareform(K)
    np.fill_diagonal(K, 1 + 3*math.e**(-7))
  else:
    dist = cdist(i/length_scale, j/length_scale, metric="sqeuclidean")
    K = np.exp(-0.5 * dist)
  return K

In [ ]:
my_model = GPR()
my_model.fit(X_train[:20000], y_train[:20000])
my_pred = my_model.predict(X_test)
display(my_pred)

mae = mean_absolute_error(y_test, my_pred)
print("Mean Absolute Error for my method: "+ str(mae))

display("Size of samples: " + str(len(y_test)))
#Variance check for two tailed test
display("Variance of Predicted Samples: " + str(np.var(np.exp(my_pred) - 1)), "Variance of Ground Truth: " + str(np.var(np.exp(y_test)) - 1))

#Ratio
display("Ratio to check for nearly equal Variance: " + str(np.var(my_pred)/np.var(y_test)))

#T-Test
tstat, t_pval = stats.ttest_ind(a=my_pred, b=y_test, equal_var=True)
display("T-Statistic for my method: " + str(tstat.round(3)), "PValue: " + str(t_pval.round(3)))

array([ 7.25378815,  0.01633581,  0.07423726, ..., -0.06799264,
        7.56143534,  8.00075456])

Mean Absolute Error for my method: 0.1304861462433534


'Size of samples: 4568'

'Variance of Predicted Samples: 16206324.238080868'

'Variance of Ground Truth: 16867028.96487056'

'Ratio to check for nearly equal Variance: 0.9899985233244486'

'T-Statistic for my method: -0.128'

'PValue: 0.898'

In [ ]:
vals = pd.DataFrame([[pd.to_datetime('2020-05-15 12:30:00'), '32.14768473',	'52.35325513',	'0.6492476293', 'HmiyD2TTLFNqkNe']], columns=('DATE_TIME', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION', 'SOURCE_KEY'))
vals['hour'] = vals['DATE_TIME'].dt.hour
vals['min'] = vals['DATE_TIME'].dt.minute
vals = vals.drop(['DATE_TIME'], axis = 1)
display(vals)
vals[cols_to_scale] = scaler.transform(vals[cols_to_scale])
vals = cyclical.transform(vals)
vals = vals.drop(['SOURCE_KEY'], axis = 1)
predicted_value = np.exp(my_model.predict(vals)) - 1
if predicted_value < 0:
  predicted_value = 0.0
display("DC_POWER Prediction for sample: " + str(predicted_value))

,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,SOURCE_KEY,hour,min
0,32.14768473,52.35325513,0.6492476293,HmiyD2TTLFNqkNe,12,30


'DC_POWER Prediction for sample: [8893.18664073]'